## 欠損値補完用データの提示
- 対象ファイル：[merged_data.csv](merged_data.csv)
- 欠損：行87-115の買取価格
- ソース：行88,93,97 → [もえたく!](https://www.netoff.co.jp/moetaku/tcg/pokemon_card/) より該当部分をpng保存
- 行88:[コレクション ムーン](88_collection_moon_box_price.png) / 行93:[闘う虹を見たか](93_tatakauniji_box_price.png) / 行97:[超次元の暴獣](97_choujigenn_box_price.png)

- それ以外の行→[ラッシュメディア](https://cardrush.media/pokemon/buying_prices?displayMode=%E3%83%AA%E3%82%B9%E3%83%88&limit=500&name=&rarity=&model_number=&amount=&page=1&sort%5Bkey%5D=amount&sort%5Border%5D=desc&associations%5B%5D=ocha_product&to_json_option%5Bexcept%5D%5B%5D=original_image_source&to_json_option%5Bexcept%5D%5B%5D=created_at&to_json_option%5Binclude%5D%5Bocha_product%5D%5Bonly%5D%5B%5D=id&to_json_option%5Binclude%5D%5Bocha_product%5D%5Bmethods%5D%5B%5D=image_source&pack_code=&element=&pack_name=&display_category=%E3%83%91%E3%83%83%E3%82%AF&is_hot%5B%5D=true&is_hot%5B%5D=false) よりwebスクレイピングで補完用データ抽出
- 保存先：[pokeka_sell_source.csv](pokeka_sell_source.csv)


#### ライブラリのインポート

In [16]:
import pandas as pd
import re

#### 元データ`merged_data.csv`の読み込みとパック名の抽出

In [17]:
# CSVファイルの読み込み
df = pd.read_csv("merged_data.csv")

# 85行目以降の「パック名」列を抽出
pack_column = df.loc[85:, "パック名"].dropna()

# 『』内の文字列を抽出
extracted_titles = pack_column.apply(
    lambda x: re.search(r"『(.*?)』", x).group(1) if re.search(r"『(.*?)』", x) else None
)

# nullを除いてリスト化
title_list = extracted_titles.dropna().tolist()

# 抽出されたパック名の確認
print(title_list)

['コレクション サン', 'コレクション ムーン', 'サン＆ムーン', 'キミを待つ島々', 'アローラの月光', '新たなる試練の向こう', '闘う虹を見たか', '光を喰らう闇', 'ひかる伝説', '覚醒の勇者', '超次元の暴獣', 'GXバトルブースト', 'ウルトラサン', 'ウルトラムーン', 'ウルトラフォース', '禁断の光', 'ドラゴンストーム', 'チャンピオンロード', '裂空のカリスマ', '迅雷スパーク', 'フェアリーライズ', '超爆インパクト', 'ダークオーダー', 'GXウルトラシャイニー', 'タッグボルト', 'ナイトユニゾン', 'フルメタルウォール', 'ダブルブレイズ', '名探偵ピカチュウ']


#### BOX買取データ`BOX_only.csv`の読み込み

In [18]:
# BOX買取データの読み込み
box_df = pd.read_csv("BOX_only.csv")

# カラム名確認
print(box_df.columns)

Index(['カード名/商品名', '買取価格'], dtype='object')


#### パック名に一致する行の抽出

In [19]:
# 対象パック名を含む行だけを抽出
filtered_df = box_df[box_df["カード名/商品名"].apply(
    lambda x: any(name in x for name in title_list))
].reset_index(drop=True)

# 必要なカラムだけに絞る
filtered_df = filtered_df[["カード名/商品名", "買取価格"]]

# 抽出結果の確認
filtered_df.head()

,カード名/商品名,買取価格
0,ハイクラスパック『GXバトルブースト』(未開封BOX),"¥500,000"
1,裂空のカリスマ(ポケモンセンター限定セット)(未開封BOX),"¥450,000"
2,強化拡張パック『新たなる試練の向こう』(未開封BOX),"¥300,000"
3,タッグボルト(ポケモンセンター限定セット)(未開封BOX),"¥230,000"
4,強化拡張パック『ひかる伝説』(未開封BOX),"¥170,000"


#### さらに特定の文字列を含む行を除外

In [20]:
# 「ポケモンセンター限定セット」を含む行を除外
final_df = filtered_df[~filtered_df["カード名/商品名"].str.contains("ポケモンセンター限定セット", na=False)]

# 結果確認
final_df.head()

,カード名/商品名,買取価格
0,ハイクラスパック『GXバトルブースト』(未開封BOX),"¥500,000"
2,強化拡張パック『新たなる試練の向こう』(未開封BOX),"¥300,000"
4,強化拡張パック『ひかる伝説』(未開封BOX),"¥170,000"
5,拡張パック『アローラの月光』(未開封BOX),"¥130,000"
6,拡張パック『裂空のカリスマ』(未開封BOX),"¥130,000"


#### CSV保存

In [22]:
final_df.to_csv("pokeka_sell_source.csv", index=False, encoding="utf-8-sig")